In [2]:
pip install streamlit pandas scikit-surprise

  Using cached streamlit-1.30.0-py2.py3-none-any.whl (8.4 MB)
  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162677 sha256=950fa537a4d4949fcfecb8093f72aba0afff4ae046378f968135e72bf63e5e9b
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [10]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import streamlit as st

# Load the dataset
file_path = "final_ratings.csv"
df = pd.read_csv(file_path)

# Create a Surprise Dataset
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)

# Build the train set
trainset = data.build_full_trainset()

# Build the SVD model
svd_model = SVD()
svd_model.fit(trainset)

# Streamlit app
st.title("Book Recommendation App")

# Get book recommendations based on input book title
input_book_title = st.text_input("Enter a book title:", "The Da Vinci Code")  # Default book title for example

# Get book ISBN based on input book title
input_book_isbn = df[df['Book-Title'] == input_book_title]['ISBN'].values

if len(input_book_isbn) > 0:
    # Predict ratings for the input book
    input_book_ratings = [(trainset.to_inner_iid(isbn), svd_model.predict(trainset.to_inner_uid(6242), trainset.to_inner_iid(isbn)).est)
                          for isbn in input_book_isbn]

    # Sort predicted ratings in descending order
    input_book_ratings = sorted(input_book_ratings, key=lambda x: x[1], reverse=True)

    st.subheader("Top 5 Recommended Books:")

    # Display top 5 recommended books
    for i in range(min(5, len(input_book_ratings))):
        book_id, predicted_rating = input_book_ratings[i]
        book_title = df[df['ISBN'] == trainset.to_raw_iid(book_id)]['Book-Title'].values[0]
        book_author = df[df['ISBN'] == trainset.to_raw_iid(book_id)]['Book-Author'].values[0]
        st.write(f"{i + 1}. {book_title} by {book_author} (Predicted Rating: {predicted_rating:.2f})")

else:
    st.warning("Book not found. Please enter a valid book title.")

